In [27]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [18]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [20]:
loss_fn = nn.CrossEntropyLoss()

In [21]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [22]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # 予測と損失の計算
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302980  [    0/60000]
loss: 2.297552  [ 6400/60000]
loss: 2.293366  [12800/60000]
loss: 2.290965  [19200/60000]
loss: 2.296827  [25600/60000]
loss: 2.279361  [32000/60000]
loss: 2.274192  [38400/60000]
loss: 2.269033  [44800/60000]
loss: 2.256593  [51200/60000]
loss: 2.263503  [57600/60000]
Test Error: 
 Accuracy: 31.9%, Avg loss: 0.035582 

Epoch 2
-------------------------------
loss: 2.246124  [    0/60000]
loss: 2.263427  [ 6400/60000]
loss: 2.245786  [12800/60000]
loss: 2.257674  [19200/60000]
loss: 2.275934  [25600/60000]
loss: 2.232781  [32000/60000]
loss: 2.224876  [38400/60000]
loss: 2.214975  [44800/60000]
loss: 2.194067  [51200/60000]
loss: 2.211834  [57600/60000]
Test Error: 
 Accuracy: 31.2%, Avg loss: 0.034887 

Epoch 3
-------------------------------
loss: 2.179569  [    0/60000]
loss: 2.220195  [ 6400/60000]
loss: 2.187092  [12800/60000]
loss: 2.211974  [19200/60000]
loss: 2.247899  [25600/60000]
loss: 2.171403  [32000/600

In [42]:
len(test_dataloader.dataset)

10000

In [43]:
import torch
import torch.onnx as onnx
import torchvision.models as models

In [44]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), "model_weights.pth")

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\pc/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100.0%


In [45]:
model = models.vgg16()
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [46]:
torch.save(model, 'model.pth')

In [47]:
model = torch.load('model.pth')